# **Day 27: Bayesian 최적화 & 하이퍼파라미터 자동 탐색**

---

## 1. Theory: 왜 Bayesian 최적화인가?

* **Grid Search/Random Search 한계**

  * 그리드: 탐색 공간이 커지면 조합 수가 폭발적으로 증가
  * 랜덤: 좋은 영역을 놓칠 가능성

* **Bayesian 최적화**

  1. **Surrogate Model**

     * 보통 **가우시안 프로세스(GP)** 또는 **트리 기반** 모델로
     * 실제 목적 함수(예: 검증 정확도)를 근사
  2. **Acquisition Function**

     * 탐색과 활용 사이의 균형을 조절
     * 대표 예: **Expected Improvement (EI)**, **Upper Confidence Bound (UCB)**
  3. **반복 루프**

     1. 초기 몇 점을 무작위로 평가
     2. Surrogate를 학습
     3. Acquisition을 최대화하는 차기 점 선택
     4. 실제 모델 평가 → 데이터에 추가 → 2단계로 돌아감
  4. **장점**

     * 제한된 평가 횟수에선 가장 유망한 후보를 뽑아 효율적으로 탐색
     * 고차원·비선형 공간에서도 강건

---

## 3. 실습 과제

1. **다른 파라미터 추가**

   * 예: `criterion=['gini','entropy']`, `max_features=['auto','sqrt']`
2. **다른 모델** 적용

   * SVM(`trial.suggest_categorical('kernel', [...])`), XGBoost 등
3. **딥러닝 모델 튜닝**

   * PyTorch MLP에서 `lr`, `dropout_p`, `weight_decay` 등을 Optuna로 자동 탐색
   * `epoch` 수, `batch_size`까지 포함해 보세요

---


In [5]:
import optuna
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris

#optuna: Bayesian 최적화 프레임워크
#load_iris: 예제용 Iris 데이터셋 로드 함수
#RandomForestClassifier: 탐색할 모델
#cross_val_score: 교차검증 점수 계산 도구

In [23]:
iris=load_iris()
x,y=iris.data, iris.target
#X: 특성 행렬, shape = (150, 4)
#y: 정답 레이블 벡터, shape = (150,)

In [15]:
def objective(train):
    n_estimator=train.suggest_int('n_estimator', 10, 200)
    max_depth=train.suggest_int('max_depth',2,20)
    min_sample_split=train.suggest_int('min_sample_split',2,10)

    clf=RandomForestClassifier(
        n_estimators=n_estimator,
        max_depth=max_depth,
        min_samples_split=min_sample_split,
        random_state=42
    )

    score=cross_val_score(clf, x,y,cv=3).mean()
    return score



objective 함수

Optuna가 “한 번의 trial(실험)”마다 평가하는 함수

trial.suggest_int(name, low, high)

이름(name)으로 파라미터를 등록하고, 구간 [low, high] 사이 정수 값을 샘플링

여기서는

n_estimator: 트리 개수 (10~200)

max_depth: 최대 깊이 (2~20)

min_sample_split: 내부 노드 분할 최소 샘플 수 (2~10)

이 파라미터들로 랜덤포레스트 모델을 만들고,

cross_val_score(..., cv=3) 로 3-폴드 교차검증 정확도를 계산 → 평균값을 score로 반환

Optuna는 objective가 반환하는 score를 최대화하려고 시도


In [61]:
study=optuna.create_study(direction='maximize')
#study.optimize(objective,n_trials=50)
study.optimize(objector,n_trials=50)


[I 2025-05-16 14:00:09,948] A new study created in memory with name: no-name-d5625eb0-2ae9-4a4f-a0d4-d6a63ecd2598
[I 2025-05-16 14:00:10,149] Trial 0 finished with value: 0.9533333333333333 and parameters: {'n_estimators': 118, 'max_depth': 3, 'min_samples_split': 2, 'criterion': 'entropy', 'max_features': 0.5}. Best is trial 0 with value: 0.9533333333333333.
[I 2025-05-16 14:00:10,214] Trial 1 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 37, 'max_depth': 17, 'min_samples_split': 3, 'criterion': 'gini', 'max_features': 0.5}. Best is trial 1 with value: 0.9666666666666667.
[I 2025-05-16 14:00:10,351] Trial 2 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 83, 'max_depth': 9, 'min_samples_split': 6, 'criterion': 'entropy', 'max_features': 'log2'}. Best is trial 1 with value: 0.9666666666666667.
[I 2025-05-16 14:00:10,480] Trial 3 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 77, 'max_depth': 10, 'min_samples_

스터디 생성 및 최적화 실행

create_study(direction='maximize')

goal: objective의 반환값(정확도)을 최대화

study.optimize(objective, n_trials=50)

objective 함수를 최대 50번 호출

각 trial마다

suggest_*로 파라미터 샘플링

모델 학습·평가

score 기록

surrogate 모델(GP 등) 업데이트 → 다음 trial에 더 좋은 후보 제안

In [28]:
print("Best Accuracy:", study.best_value)
print("Best Params:")
for key, val in study.best_trial.params.items():
    print(f"  {key}: {val}")

'''
결과 조회
study.best_value: 최적 trial이 낸 최고 점수(교차검증 정확도)
study.best_trial.params: 최적 trial에서 선택된 파라미터 조합

'''

Best Accuracy: 0.9666666666666667
Best Params:
  n_estimator: 69
  max_depth: 16
  min_sample_split: 8


'\n결과 조회\nstudy.best_value: 최적 trial이 낸 최고 점수(교차검증 정확도)\nstudy.best_trial.params: 최적 trial에서 선택된 파라미터 조합\n\n'


### 전체 흐름 요약

1. **데이터 준비** → 2. **objective 함수 정의**
2. **파라미터 공간**을 `suggest_*`로 설정
3. `cross_val_score`로 모델 성능 측정
4. **Optuna 스터디** 생성 → `optimize()` 호출

   * surrogate → acquisition → 파라미터 제안 → 평가 → 업데이트
5. **최적값**과 **최적 파라미터**를 `study.best_value`/`study.best_trial.params`로 확인

이 과정을 통해 전통적인 Grid/Random Search보다 훨씬 적은 실험 횟수로, 높은 성능을 내는 파라미터 조합을 효율적으로 찾을 수 있습니다.


In [59]:
def objector(trial):
    # 기존 파라미터
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth    = trial.suggest_int('max_depth', 2, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)

    # 추가 탐색 파라미터
    criterion    = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_features = trial.suggest_categorical('max_features',
                                             ['sqrt', 'log2', 0.5,None])

    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        criterion=criterion,
        max_features=max_features,
        random_state=42
    )

    score = cross_val_score(clf, x, y, cv=3).mean()
    return score


In [47]:
#다른 파라미터 추가하기
def objector(train):
    n_estimator=train.suggest_int('n_estimator', 10, 200)
    max_depth=train.suggest_int('max_depth',2,20)
    min_sample_split=train.suggest_int('min_sample_split',2,10)
    
    criter=train.suggest_categorical('criterion', ['gini', 'entropy'])
    max_feature=train.suggest_categorical('max_features',
                                             ['auto', 'sqrt', 'log2', 0.5])

    clf=RandomForestClassifier(
        n_estimators=n_estimator,
        max_depth=max_depth,
        min_samples_split=min_sample_split,
        criterion=criter,
        max_features=max_feature,
        random_state=42
    )

    score=cross_val_score(clf, x,y,cv=3).mean()
    return score

# 다른 모델 적용하기

In [77]:
#SVM 분류기
from sklearn.svm import SVC

def objector_svc(trial):
    C=trial.suggest_loguniform("C",1e-3, 1e3)
    kernel=trial.suggest_categorical('kernel',['linear', 'rbf', 'poly'])
    gamma=trial.suggest_loguniform('gamma',1e-4, 1e-1)

    clf=SVC(C=C,kernel=kernel,gamma=gamma,probability=True)

    score=cross_val_score(clf,x,y,cv=3).mean()
    return score


study_svm = optuna.create_study(direction='maximize')
study_svm.optimize(objector_svc, n_trials=30)

#suggest_loguniform은 로그 스케일 탐색에 유용합니다.
#probability=True를 넣어야 predict_proba를 쓸 때 오류가 없습니다.

[I 2025-05-16 14:10:39,957] A new study created in memory with name: no-name-3e6e571e-eb5f-405d-b55a-1a239005a0df
C:\Users\JH\AppData\Local\Temp\ipykernel_39732\1590068917.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C=trial.suggest_loguniform("C",1e-3, 1e3)
C:\Users\JH\AppData\Local\Temp\ipykernel_39732\1590068917.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma=trial.suggest_loguniform('gamma',1e-4, 1e-1)
[I 2025-05-16 14:10:39,964] Trial 0 finished with value: 0.98 and parameters: {'C': 75.04826342309957, 'kernel': 'rbf', 'gamma': 0.003190074554097529}. Best is trial 0 with value: 0.98.
C:\Users\JH\AppData\Local\Temp\ipykernel_39732\1590068917.py:5: F

In [88]:
#XGBoost 분류기
import xgboost as xgb

def objective_xgb(trial):
    param={
        'objective':'multi:softprob',
        'num_class':3,
        'eta':trial.suggest_loguniform('eta', 1e-3, 1e-1),
        'max_depth':trial.suggest_int('max_depth',3,10),
        'subsample':trial.suggest_float('subsample', 0.5, 1.0),
        'colsample':trial.suggest_float('colsample',0.5, 1.0)
    }

    dtrain=xgb.DMatrix(x,label=y)
    cv=xgb.cv(param,dtrain,num_boost_round=100,nfold=3,metrics='mlogloss',early_stopping_rounds=10)
    return -cv['test-mlogloss-mean'].min()


study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=30)

[I 2025-05-16 14:24:14,646] A new study created in memory with name: no-name-d07c7c75-d674-4f7e-8e10-5bf3ac8bd1ea
C:\Users\JH\AppData\Local\Temp\ipykernel_39732\1726104957.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'eta':trial.suggest_loguniform('eta', 1e-3, 1e-1),
C:\Users\JH\anaconda\Lib\site-packages\xgboost\training.py:209: UserWarning: [14:24:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "colsample" } are not used.

  return getattr(self.bst, name)(*args, **kwargs)
C:\Users\JH\anaconda\Lib\site-packages\xgboost\training.py:215: UserWarning: [14:24:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "colsample" } are not used.

  self.bst.update(self.dtrain, iteration, fobj)
[I 2025-05-16 14:24:14,827] Trial 0 finished with value: -0.8853

# 딥러닝(PyTorch) 모델 튜닝하기

In [91]:
import torch
import torch.nn as nn
import torch.optim as optim

class MLP(nn.Module):
    def __init__(self, hidden_size, dropout_p):
        super().__init__()
        self.fc1 = nn.Linear(28*28, hidden_size)
        self.drop = nn.Dropout(dropout_p)
        self.fc2 = nn.Linear(hidden_size, 10)
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.drop(x)
        return self.fc2(x)

In [101]:
def object_torch(train):
    hidden=train.suggest_int('hidden', 64, 512, step=64)
    drop=train.suggest_float('drop',0.0, 0.5)
    lr=train.suggest_loguniform('lr',1e-4, 1e-1)
    weight_decay=train.suggest_loguniform('weight_decay', 1e-6, 1e-2)

    model=MLP(hidden,drop).to(device)
    criter=nn.CrossEntropyLoss()
    optimy=optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay
    )

    model.train()
    data,target=next(iter(train_loader))
    data,target=data.to(device),target.to(device)
    optimy.zero_grad()
    out = model(data)
    loss = criter(out, target)
    loss.backward()
    optimy.step()
    pred = out.argmax(1)
    acc = (pred == target).float().mean().item()

    return acc

In [107]:
device=torch.device('cuda'if torch.cuda.is_available()else'cpu')
study_torch = optuna.create_study(direction='maximize')
study_torch.optimize(object_torch, n_trials=20)

[I 2025-05-17 02:43:35,386] A new study created in memory with name: no-name-dd89d6e0-9771-4838-b9d8-ba5c98ffaa31
C:\Users\JH\AppData\Local\Temp\ipykernel_39732\2543822558.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr=train.suggest_loguniform('lr',1e-4, 1e-1)
C:\Users\JH\AppData\Local\Temp\ipykernel_39732\2543822558.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay=train.suggest_loguniform('weight_decay', 1e-6, 1e-2)
[W 2025-05-17 02:43:38,903] Trial 0 failed with parameters: {'hidden': 192, 'drop': 0.457096911118207, 'lr': 0.0009827088790178623, 'weight_decay': 2.5783602426486673e-05} because of the following error: NameError("name 'train_loader

NameError: name 'train_loader' is not defined